In [ ]:
!pip install diffusers
!pip install transformers

In [ ]:
import torch
from torch import optim

from train import train, save
from model import VaReSynth
from demo import demo

# Create the model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.manual_seed(0)

model = VaReSynth().to(device)
# model_ema = deepcopy(model)
print('Model parameters:', sum(p.numel() for p in model.parameters()))

opt = optim.Adam(model.parameters(), lr=2e-4)
scaler = torch.cuda.amp.GradScaler()

# Use a low discrepancy quasi-random sequence to sample uniformly distributed
# timesteps. This considerably reduces the between-batch variance of the loss.
rng = torch.quasirandom.SobolEngine(1, scramble=True)

epoch = 0
while True:
    train(model, opt, scaler, rng, epoch)
    save(model, opt, scaler, epoch)
    epoch += 1
    if epoch % 5 == 0:
        demo(model, epoch)